In [ ]:
import os
import ase.io
import re
import pandas as pd
import numpy as np
from dscribe.descriptors import SOAP
from dscribe.descriptors import MBTR,ACSF,CoulombMatrix,EwaldSumMatrix,SineMatrix 
import tensorflow as tf
from tensorflow.keras.models import Model
from numpy                 import array
from sklearn               import metrics
from sklearn.preprocessing import MinMaxScaler
from keras.models          import Sequential
from keras.layers          import *
from sklearn               import metrics
from tensorflow.keras import layers
from tcn import TCN
import keras
from keras.layers import Conv1D, MaxPooling1D, Dense, Flatten,Conv2D, MaxPool2D,LSTM,Bidirectional
from keras.models import Sequential
from keras import Input
from keras.layers          import *
from keras.models import save_model,load_model
from numpy import save,load
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import random

In [ ]:
"""
MSE  ：均方误差    ----->  预测值减真实值求平方后求均值
RMSE ：均方根误差  ----->  对均方误差开方
MAE  ：平均绝对误差----->  预测值减真实值求绝对值后求均值
R2   ：决定系数，可以简单理解为反映模型拟合优度的重要的统计量
"""
def compute_metrics(pred,real):
    MSE   = metrics.mean_squared_error(pred, real)
    RMSE  = metrics.mean_squared_error(pred, real)**0.5
    MAE   = metrics.mean_absolute_error(pred, real)
    R2    = metrics.r2_score(pred, real)
    print('均方误差: %.5f' % MSE)
    print('均方根误差: %.5f' % RMSE)
    print('平均绝对误差: %.5f' % MAE)
    print('R2: %.5f' % R2)
    return [MSE,RMSE,MAE,R2]
seed=1
os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
######specify the input and output######
data_input=np.load('data_input_sm_ew_acsf_rcu6.npy')
data_output=np.load('data_output_sm_ew_acsf_rcu6_SLAB.npy')

In [ ]:
# 建构模型
def set_up_model(model_type,X_train,X_test,y_train,y_test):
    if model_type == 1:
        # 双向 LSTM
        model = Sequential()
        model.add(Bidirectional(LSTM(100),
                                input_shape=(X_train.shape[1],X_train.shape[2])))
        model.add(Dense(y_train.shape[1]))
    if model_type == 2: 
        # simple RNN
        model = Sequential()
        model.add(SimpleRNN(units=100, return_sequences=True,activation='relu',
                       input_shape=(X_train.shape[1],X_train.shape[2])))
        model.add(SimpleRNN(units=100))
        model.add(Dense(y_train.shape[1]))
    if model_type == 3:
    #     GRU
        model = Sequential()
        model.add(GRU(units=100, return_sequences=True,activation='relu',
                       input_shape=(X_train.shape[1],X_train.shape[2])))
        model.add(GRU(units=100))
        model.add(Dense(y_train.shape[1]))
    if model_type == 4:
        #simple one layer CNN
        model = Sequential()
        model.add(Conv1D(filters=32, kernel_size=3, activation='relu', 
                         input_shape=(X_train.shape[1],X_train.shape[2])))

        model.add(MaxPooling1D(pool_size=2))
        model.add(Flatten())
        model.add(Dense(10, activation='relu'))
        model.add(Dense(units=y_train.shape[1]))
    if model_type == 5:
        #multilayer CNN
        model = Sequential()
        model.add(Conv1D(filters=64, kernel_size=3, activation='relu', 
                         input_shape=(X_train.shape[1],X_train.shape[2])))
        model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Flatten())
        model.add(Dense(100, activation='relu'))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(units=y_train.shape[1]))
    if model_type == 6:
        #CNN-LSTM
#         y_train_sp = y_train.reshape((y_train.shape[0], y_train.shape[1], 1))
#         y_test_sp = y_test.reshape((y_test.shape[0], y_test.shape[1], 1))
        model = Sequential()
        model.add(Conv1D(filters=64, kernel_size=3, activation='relu',
                         input_shape=(X_train.shape[1],X_train.shape[2])))
        model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Flatten())
        model.add(RepeatVector(y_train.shape[1]))
        model.add(LSTM(200, activation='relu', return_sequences=True))
        model.add(TimeDistributed(Dense(100, activation='relu')))
        model.add(TimeDistributed(Dense(y_train.shape[1])))

    if model_type == 7:
        #单层TCN
        model = Sequential([
        TCN(input_shape=(X_train.shape[1],X_train.shape[2]),
            nb_filters=64,
            kernel_size=2,
            nb_stacks=1,
            dilations=(1, 2, 4, 8, 16,32,64,128,256,512),
            padding='causal',
            use_skip_connections=True,
            dropout_rate=0.0,
            return_sequences=False,
            activation='relu',
            kernel_initializer='he_normal',
            use_batch_norm=False,
            use_layer_norm=False,
            use_weight_norm=False,
            ),
        Dense(y_train.shape[1], activation='linear')
    ])
    if model_type == 8:
        #多层TCN
        model = Sequential([
        TCN(input_shape=(X_train.shape[1],X_train.shape[2]),
            nb_filters=64,
            kernel_size=2,
            nb_stacks=1,
            dilations=(1, 2, 4, 8, 16,32,64,128,256,512),
            padding='causal',
            use_skip_connections=True,
            dropout_rate=0.0,
            return_sequences=True,
            activation='relu',
            kernel_initializer='he_normal',
            use_batch_norm=False,
            use_layer_norm=False,
            use_weight_norm=False,
            ),
        TCN(
            return_sequences=False
            ),
        Dense(y_train.shape[1], activation='linear')
    ])
    if model_type == 9:
        model = Sequential()
        model.add(Conv1D(filters=128, kernel_size=2, activation='relu', 
                         input_shape=(X_train.shape[1],X_train.shape[2])))
        model.add(Conv1D(filters=64, kernel_size=2, activation='relu'))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Conv1D(filters=32, kernel_size=2, activation='relu'))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Flatten())
        model.add(Dense(100, activation='relu'))
        model.add(Dense(units=y_train.shape[1]))
    if model_type == 10:  
        model = Sequential()
        # 表示我们的网络将学习16个滤波器 每个滤波器的大小都是5×5，步长为1
        model.add(Conv2D(32, kernel_size=2, strides=1, padding='valid', input_shape=(X_train.shape[1],X_train.shape[2],1),activation="relu"))
        # 2×2的最大池化层 步长为2
        model.add(MaxPool2D(pool_size=2, strides=2))
        model.add(Conv2D(16, kernel_size=2, strides=1, padding='valid',activation="relu"))
        # 2×2的最大池化层 步长为2
        model.add(MaxPool2D(pool_size=2, strides=2))
        # 展开
        model.add(Flatten())
        # 接下来相当于有两层full-connected网络
        # 120个神经元 全连接网络
        model.add(Dense(100,activation="relu"))
        # model.add(Dense(50,activation="relu"))
        # 84个神经元 全连接网络
        model.add(Dense(y_train.shape[1],activation="linear"))
    return model


In [ ]:
def train_model(model_type,model,number_batchsize,n_epochs,X_train,X_test,y_train,y_test):
    print('type of model is',model_type)

    history = model.fit(X_train, y_train, 
                        batch_size=number_batchsize, 
                        epochs=n_epochs, 
                        validation_data=(X_test, y_test), 
                        validation_freq=1,verbose=0)                  #测试的epoch间隔数



In [ ]:
def prediction(model_type,model,Input):
    pred=model.predict(Input)
    if model_type in [8,9]:
        pred=pred.reshape(pred.shape[0],pred.shape[1])
    return pred

In [ ]:
def compute_MAE_within_percent(pred_test,real_test,pred_train,real_train,threshold):
    difference_test=abs(pred_test-real_test)
    difference_train=abs(pred_train-real_train)
    i_test=0
    i_train=0
    for each in difference_test:
        if each<threshold:
            i_test+=1
    for each in difference_train:
        if each<threshold:
            i_train+=1
    percent_train=i_train/len(difference_train)
    percent_test=i_test/len(difference_test)
    return percent_train,percent_test

In [ ]:
def evaluate_and_save(predict_target,model_type,X_train,X_test,y_train,y_test,lr,loss_type,batch_number,epoch_number,Outputscaler):
    print('now model type is',model_type)
    #####build up model and train######
    current_model=set_up_model(model_type,X_train,X_test,y_train,y_test)
    current_model.compile(optimizer=tf.keras.optimizers.Adam(lr),loss=loss_type)  # 损失函数用均方误差
    train_model(model_type,current_model,batch_number,epoch_number,X_train,X_test,y_train,y_test)
    TRAIN_SCORES=[]
    TEST_SCORES=[]
    #####give predictions#####
    try:
        if model_type==6:
            pred_test=prediction(model_type,current_model,X_test)
            pred_train=prediction(model_type,current_model,X_train)

            pred_test=pred_test.reshape(-1,1)
            pred_train=pred_train.reshape(-1,1)

            pred_train=Outputscaler.inverse_transform(pred_train)
            pred_test=Outputscaler.inverse_transform(pred_test)

            print('here')

            real_train=Outputscaler.inverse_transform(y_train.reshape(-1,1))
            real_test=Outputscaler.inverse_transform(y_test.reshape(-1,1))

        else:
            pred_test=prediction(model_type,current_model,X_test)
            pred_train=prediction(model_type,current_model,X_train)

            pred_train=Outputscaler.inverse_transform(pred_train)
            pred_test=Outputscaler.inverse_transform(pred_test)

            real_train=Outputscaler.inverse_transform(y_train)
            real_test=Outputscaler.inverse_transform(y_test)

        TRAIN_SCORES=compute_metrics(pred_train,real_train)
        TEST_SCORES=compute_metrics(pred_test,real_test)
        fig=plt.figure()
        ax = fig.add_subplot(111)
        x_y_x=np.arange(-865,-845,0.01)
        x_y_y=np.arange(-865,-845,0.01)
        ax.scatter(pred_train,real_train,c='blue',label='Train',alpha=0.25)
        ax.scatter(pred_test,real_test,c='red',label='Test',alpha=0.75)
        ax.plot(x_y_x,x_y_y,c='black')
        plt.legend()
        plt.xlabel('Predicted_SLAB Energy (eV)')
        plt.ylabel('Real_SLAB Energy (eV')
        plt.show()
    except Exception as e:
        print(e)
    return current_model,TRAIN_SCORES,TEST_SCORES

In [ ]:
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split
Minmaxsc  = MinMaxScaler(feature_range=(0, 1))
Minmaxsc2  = MinMaxScaler(feature_range=(0, 1))
Stdsc  = StandardScaler()
Stdsc2  = StandardScaler()
MAsc  = MaxAbsScaler()
MAsc2  = MaxAbsScaler()
Rsc  = RobustScaler()
Rsc2  = RobustScaler()

X = Minmaxsc.fit_transform(data_input.reshape(-1, data_input.shape[-1])).reshape(data_input.shape)
y = Rsc2.fit_transform(data_output.reshape(-1,1))
random_seed=353
X_train, X_test, y_train, y_test=train_test_split(X,y,random_state=random_seed,test_size=0.1)
X_train = X_train.reshape(-1, X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(-1, X_train.shape[1], X_train.shape[2], 1)



In [ ]:
serial_model.save('CNN.h5')

In [ ]:
serial_model=load_model('CNN.h5')

In [ ]:
pred_test=prediction(9,serial_model,X_test)
pred_train=prediction(9,serial_model,X_train)

pred_train=Rsc2.inverse_transform(pred_train)
pred_test=Rsc2.inverse_transform(pred_test)

real_train=Rsc2.inverse_transform(y_train)
real_test=Rsc2.inverse_transform(y_test)

TRAIN_SCORES=compute_metrics(pred_train,real_train)
TEST_SCORES=compute_metrics(pred_test,real_test)
fig=plt.figure()
ax = fig.add_subplot(111)
x_y_x=np.arange(-860,-845,0.01)
x_y_y=np.arange(-860,-845,0.01)
ax.scatter(pred_train,real_train,c='blue',label='Train',alpha=0.25)
ax.scatter(pred_test,real_test,c='red',label='Test',alpha=0.75)
ax.plot(x_y_x,x_y_y,c='black')
plt.legend()
plt.xlabel('Predicted_SLAB Energy (eV)')
plt.ylabel('Real_SLAB Energy (eV')
plt.show()